In [1]:
import spiceypy
spiceypy.tkvrsn('TOOLKIT')

import numpy as np
import pickle
import spiceypy as spice
import os
import sys
import importlib
from datetime import timedelta
# Get file of the script and add into sys
sys.path.insert(0, "/home/mglos/skola/DIP/LavaTubeSniffer")
from src.scripts.SPICE import utils

importlib.reload(utils)

sweep_iterator = utils.SweepIterator()

computation_start = sweep_iterator.min_loaded_time
computation_end = sweep_iterator.max_loaded_time

computation_timedelta = timedelta(days=1000)
computation_now = computation_start + computation_timedelta

sweep_iterator.initiate_sweep(computation_now)

Loading lone SPICE kernels from files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 142.65it/s]


In [2]:
filename = "/media/mglos/HDD_8TB1/LOLA_DSK/{}/output7_5_percent.{}"

In [2]:
import numpy as np
import pandas as pd
from astropy.coordinates import SphericalRepresentation
import astropy.units as u

# --- Read and transform the data ---
xyz_file = filename.format("xyz", "xyz")

df = pd.read_csv(xyz_file, sep=" ", names=["x", "y", "z"])
x = df["x"].to_numpy()
y = df["y"].to_numpy()
z = (df["z"] / 100).to_numpy().astype(np.float32)  # elevation in meters

moon_radius = 1737.400  # Mean lunar radius (Km)

# Map x to longitude and y to latitude
lon = np.interp(x, (x.min(), x.max()), (-180, 180)) * u.deg
lat = (90 - (y - y.min())/(y.max()-y.min()) * 180) * u.deg
lat = -lat  # Invert latitude

# Compute the 3D radius and convert to Cartesian
radius = (moon_radius + z) * u.m
spherical_coords = SphericalRepresentation(lon, lat, radius)
cartesian_coords = spherical_coords.to_cartesian()

batch_rects = np.column_stack([
    cartesian_coords.x.value,
    cartesian_coords.y.value,
    cartesian_coords.z.value
]).astype(np.float32)

# --- Reshape grid and downsample based on latitude ---
unique_x = np.unique(x)
unique_y = np.unique(y)
num_cols = len(unique_x)
num_rows = len(unique_y)
assert num_rows * num_cols == len(x), "Data does not form a complete grid."

# Reshape to 2D grid (rows = latitudes, cols = longitudes)
vertices_grid = batch_rects.reshape(num_rows, num_cols, 3)
lon_grid = lon.value.reshape(num_rows, num_cols)
lat_grid = lat.value.reshape(num_rows, num_cols)

global_vertices_list = []  # list of downsampled row vertex arrays
row_data = []              # list of dicts: { 'indices': global indices, 'lons': sampled longitudes }
global_index = 0

for i in range(num_rows):
    # Use the row’s first latitude to compute density
    row_lat = lat_grid[i, 0]
    factor = np.cos(np.deg2rad(abs(row_lat)))
    desired_count = max(1, int(np.round(num_cols * factor)))
    col_indices = np.linspace(0, num_cols - 1, desired_count, dtype=int)
    
    row_vertices = vertices_grid[i, col_indices, :]
    row_lons = lon_grid[i, col_indices]
    indices = np.arange(global_index, global_index + len(col_indices))
    global_index += len(col_indices)
    
    global_vertices_list.append(row_vertices)
    row_data.append({'indices': indices, 'lons': row_lons})

vertices_global = np.concatenate(global_vertices_list, axis=0)
# vertices_global: (total_points, 3)

# --- Stitch adjacent rows into triangles ---
def stitch_rows(idxA, idxB, lonA, lonB):
    """Merge two rows of vertices into triangles.
       idxA, idxB: arrays of global vertex indices for the two rows.
       lonA, lonB: corresponding longitudes.
    """
    tris = []
    i, j = 0, 0
    # Advance through both rows until one is exhausted.
    while i < len(idxA)-1 and j < len(idxB)-1:
        # Compare angular gaps to decide which edge to advance.
        diffA = abs(lonA[i+1] - lonB[j])
        diffB = abs(lonB[j+1] - lonA[i])
        if diffA < diffB:
            tris.append([idxA[i], idxB[j], idxA[i+1]])
            i += 1
        else:
            tris.append([idxA[i], idxB[j], idxB[j+1]])
            j += 1
    # Append remaining triangles if one row still has vertices.
    while i < len(idxA)-1:
        tris.append([idxA[i], idxB[-1], idxA[i+1]])
        i += 1
    while j < len(idxB)-1:
        tris.append([idxA[-1], idxB[j], idxB[j+1]])
        j += 1
    return tris

triangles = []  # list of triangles (each is a list of 3 global vertex indices)
for r in range(len(row_data) - 1):
    idxA = row_data[r]['indices']
    idxB = row_data[r+1]['indices']
    lonA = row_data[r]['lons']
    lonB = row_data[r+1]['lons']
    
    # Handle pole-like cases: if one row has just one vertex.
    if len(idxA) == 1:
        for j in range(len(idxB)-1):
            triangles.append([idxA[0], idxB[j], idxB[j+1]])
    elif len(idxB) == 1:
        for i in range(len(idxA)-1):
            triangles.append([idxA[i], idxB[0], idxA[i+1]])
    else:
        triangles.extend(stitch_rows(idxA, idxB, lonA, lonB))

# vertices_global holds the point coordinates.
vertices = vertices_global
triangles = [(x+1, y+1, z+1) for x, y, z in triangles]
# triangles is a list of connectivity index triplets.


In [10]:
with open(filename.format("CACHE", "triangles_vertices_cachse.pkl"), "wb") as f:
    pickle.dump((triangles, vertices), f)

In [3]:
# import numpy as np
# import plotly.graph_objects as go

# def plot_3d_planet(points, cube=False):
#     # if cube:
#     #     # Recenter and scale to fit within a [-1000, 1000] cube
#     pickle.dump((triangles, vertices), f)
#     fig = go.Figure(data=[go.Scatter3d(
#         x=x, y=y, z=z,
#         mode='markers',
#         marker=dict(
#             size=2,
#             color=np.linalg.norm(points, axis=1),
#             colorscale='Viridis',
#             opacity=0.8
#         )
#     )])
    
#     scene_conf = dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z',
#         aspectmode='cube'
#     )
    
#     if cube:
#         scene_conf.update({
#             'xaxis': dict(range=[-2000, 2000]),
#             'yaxis': dict(range=[-2000, 2000]),
#             'zaxis': dict(range=[-2000, 2000])pickle.dump((triangles, vertices), f)is=1)

# # Example usage with cube option enabled:
# plot_3d_planet(t[:10000], cube=True)

In [3]:
with open(filename.format("CACHE", "triangles_vertices_cachse.pkl"), "rb") as f:
    triangles, vertices = pickle.load(f)

In [4]:
vertices = np.array(vertices)
triangles = np.array(triangles)
len(vertices), len(triangles), np.array(triangles).max(), np.array(triangles).min()

(15203120, 30399328, 15203120, 1)

In [5]:
# #### This is used for rect model #####
# # 🔹 Voxel Grid & Spatial Index Parameters (Adjusted to Real Values)
# FINSCL = 0.4  # Fine voxel scale
# CORSCL = 2  # Coarse voxel scale
# WORKSZ = 700_000_000
# VOXPSZ = 2_000_000_000 # Voxel-plate pointer array size
# VOXLSZ = 400_000_000  # Voxel-plate list array size
# SPXISZ = 1_400_000_000  # Spatial index size
# MAKVTL = True


FINSCL = 6.9  # Fine voxel scale
CORSCL = 9  # Coarse voxel scale
WORKSZ = 400_000_000
VOXPSZ = 45_000_000 # Voxel-plate pointer array size
VOXLSZ = 135_000_000  # Voxel-plate list array size
SPXISZ = 1_250_000_000  # Spatial index size
MAKVTL = True


dsk_output = filename.format("dsk", "new_gen.dsk")

# 🔹 STEP 3: OPEN DSK FILE FOR WRITING
if os.path.exists(dsk_output):
    os.remove(dsk_output)  # Remove existing file to avoid conflicts

mncor1, mxcor1 = np.min(vertices[:, 0]), np.max(vertices[:, 0])
mncor2, mxcor2 = np.min(vertices[:, 1]), np.max(vertices[:, 1])
mncor3, mxcor3 = np.min(vertices[:, 2]), np.max(vertices[:, 2])

print(f"🔹 Adjusted Bounds:")
print(f"  X: {mncor1} to {mxcor1}")
print(f"  Y: {mncor2} to {mxcor2}")
print(f"  Z: {mncor3} to {mxcor3}")

NVXTOT = ((mxcor1 - mncor1) / FINSCL) * ((mxcor2 - mncor2) / FINSCL) * ((mxcor3 - mncor3) / FINSCL)
print(f"Estimated Fine Voxel Count (NVXTOT): {NVXTOT}")

handle = spice.dskopn(dsk_output, "Generated DSK File", 0)  # Open DSK file
print(f"✅ Opened DSK file: {dsk_output}")

# 🔹 STEP 4: COMPUTE SPATIAL INDEX
print("Computing spatial index...")
spaixd, spaixi = spice.dskmi2(
    vertices,  # Vertex count & coordinates
    triangles,      # Plate count & connectivity
    FINSCL, CORSCL,        # Fine & coarse voxel scales
    WORKSZ, VOXPSZ, VOXLSZ,# Spatial index array sizes
    MAKVTL, SPXISZ         # Vertex-plate association flag & spatial index size
)
print("✅ Spatial index computed.")


# 🔹 STEP 6: WRITE DSK SEGMENT
print("Writing DSK file...")
time_start = -1e9  # Arbitrary large negative time (valid for all time)
time_end = 1e9  # Arbitrary large positive time

CORSYS = 3 # Rectangular 
#CORSYS = 4 # Planetocentric 
CORPAR = np.zeros(10) # for rectangular, unused
#CORPAR[0] = 1737.4 # Moon radius
#CORPAR[1] = 0.0 # Flattening coeffictient `corpar[0] * ( 1 - corpar[1] )``
DCLASS = 2 # Triangular mesh?? I believeNVXTOT

spice.dskw02(
    handle, 301, 1001, DCLASS, "MOON_ME", CORSYS, CORPAR,
    mncor1, mxcor1, mncor2, mxcor2, mncor3, mxcor3, time_start, time_end,
    vertices, triangles, spaixd, spaixi
)
print(f"✅ DSK segment written to {dsk_output}")

# 🔹 STEP 7: CLOSE DSK FILE
spice.dskcls(handle, True)  # Enable compression & close the file
print("✅ DSK file closed.")

#### This is used for rect model END #####


🔹 Adjusted Bounds:
  X: -1741.818603515625 to 1737.08447265625
  Y: -1740.106201171875 to 1734.867431640625
  Z: -1739.318115234375 to 1737.756591796875
Estimated Fine Voxel Count (NVXTOT): 127955981.52481353
✅ Opened DSK file: /media/mglos/HDD_8TB1/LOLA_DSK/dsk/output7_5_percent.new_gen.dsk
Computing spatial index...
✅ Spatial index computed.
Writing DSK file...
✅ DSK segment written to /media/mglos/HDD_8TB1/LOLA_DSK/dsk/output7_5_percent.new_gen.dsk
✅ DSK file closed.


In [7]:
vertices.max()

1737.3486

In [21]:
vertices.min()

-1740.4907

In [6]:
np.array(triangles).min()

1

In [32]:
triangles[0]

[0, 1, 2]